In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
import os
from config.config import config

def get_model(model:str="gpt-4o-mini", api_key:str|None=None):
    if api_key is None:
        os.environ["OPENAI_API_KEY"] = config.get("apikey", "openai")
    model = ChatOpenAI(model=model)
    return model

In [ ]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool
def get_current_time(timezone:str, location:str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존(예: 'Asia/Seoul'). 실제 존재해야 함
        location (str): 지역명. 타임존은 모든 지명에 대응되지 않으므로 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f"{timezone} ({location}) 현재 시각 : {now}"
    return location_and_local_time

In [ ]:
# 모델 선언
model = get_model()

# 도구 리스트와 딕셔너리  
tools = [get_current_time, ]
tool_dict = {"get_current_time": get_current_time, }

# 도구를 모델에 반영
llm_with_tools = model.bind_tools(tools)

In [ ]:
# messages
messages = [
    SystemMessage("당신은 사용자의 질문에 답변하기 위해 tools를 사용할 수 있습니다."),
    HumanMessage("하와이와 부산은 지금 몇시잉교?")
]

# llm_with_tools 를 사용해 사용자 질문에 대한 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# 생성된 답변 출력
print(messages)

In [ ]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

In [ ]:
llm_with_tools.invoke(messages)

In [ ]:
from pydantic import BaseModel, Field
import yfinance as yf

# stock history input schema
class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예:APPL)"),
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")

# 주식 이력 조회 tool
@tool
def get_yf_stock_history(stock_history_input:StockHistoryInput):
    """ 주식 종목의 가격 데이터를 조회하는 함수 """
    stock = yf.Ticker(stock_history_input.ticker)
    hist = stock.history(stock_history_input.period)
    return str(hist.to_markdown())

# tool 리스트에 추가
tools = [get_current_time, get_yf_stock_history]
tool_dict = {
    "get_current_time":get_current_time,
    "get_yf_stock_history":get_yf_stock_history
}

# LLM 모델에 도구 등록
llm_with_tools = model.bind_tools(tools)

# 메시지 작성
messages.append(HumanMessage("삼성전자의 주식은 한달 전에 비해 올랐어? 내렸어?"))

# 응답 요청
response = llm_with_tools.invoke(messages)
messages.append(response)

# 응답에 오는 툴 콜을 처리
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

# 최종 응답 요청
llm_with_tools.invoke(messages)